<h3>Projects funded from USA (NSF)</h3>

<p>The National Science Foundation (NSF) funds research and education in science and engineering, through grants, contracts, and cooperative agreements. The Foundation accounts for about 20 percent of federal support to academic institutions for basic research. 

Information about research projects that NSF has funded since 1989 can be found by searching the Award Abstracts database (https://www.nsf.gov/awardsearch/download.jsp). The information includes abstracts that describe the research, and names of principal investigators and their institutions. The database includes both completed and in-process research.</p>

<p>We will use all the provided information from 1994 till 2017 (259106 awards). The dataset consists of .zip folders for each year, with .xml files for each award.</p>

In [1]:
import os
import zipfile
import cjson
import pandas as pd
import numpy as np
import xmltodict, json
import pickle

<p>First, we unzip each folder and then for each file of the folder, we convert the xml file to a dictionary with:
<ul>
<li>key: the file name/award id (ex. 9601223)</li>
<li>value: the actual content of the xml file in JSON format</li>
</ul></p>

In [2]:
from xml.dom.minidom import parse

filenames = os.listdir('USA_data')

raw_text = {}

for zipfilename in filenames:
    with zipfile.ZipFile('USA_data/'+zipfilename) as z:
        for filename in z.namelist():
            if not os.path.isdir(filename):
                try:
                    with z.open(filename) as f:
                        raw_text[filename[:-4]] = json.dumps(xmltodict.parse(f))
                except:
                    print filename
                    pass

<p>For each award/key of our dictionary, we create a new dictionary to keep the values that we'll use later on the analysis:
<ul>
<li>id: the award id</li>
<li>title: the award title</li>
<li>objective: the abstract narration of the award</li>
<li>state: the code of the state</li>
<li>subjects: information provided from 'Program Reference' and 'Program Element' values</li>
<li>foa: FoaInformation</li>
</ul>
<br>
Then, we append each dictionary to a list.
</p>

In [3]:
usa_fp4 = []
usa_fp5 = []
usa_fp6 = []
usa_fp7 = []
usa_h2020 = []

for key, val in raw_text.items():
    usa_dict = {}

    value = cjson.decode(val)
    new_val = value['rootTag']['Award']

    usa_dict['id'] = new_val['AwardID']
    usa_dict['title'] = new_val['AwardTitle']
    usa_dict['objective'] = new_val['AbstractNarration']
    usa_dict['state'] = new_val['Institution']['StateCode']
    
    if key[:2] in ['94', '95', '96', '97']:
        usa_dict['framework_programme'] = 'FP4'
    elif key[:2] in ['98', '99', '00', '01']:
        usa_dict['framework_programme'] = 'FP5'
    elif key[:2] in ['02', '03', '04', '05', '06']:
        usa_dict['framework_programme'] = 'FP6'
    elif key[:2] in ['07', '08', '09', '10', '11', '12', '13']:
        usa_dict['framework_programme'] = 'FP7'
    else:
        usa_dict['framework_programme'] = 'H2020'
    
    try:
        if type(new_val['ProgramReference']) is dict:
            usa_dict['subjects'] = new_val['ProgramElement']['Text'] + new_val['ProgramReference']['Text']
        elif type(new_val['ProgramReference']) is list:
            text = ''
            for i in new_val['ProgramReference']:
                text = text + ' ' + i['Text']

            usa_dict['subjects'] = text
        else:
            pass
    except:
        pass


    try:
        if type(new_val['FoaInformation']) is dict:
            usa_dict['foa'] = new_val['FoaInformation']['Name']
        elif type(new_val['FoaInformation']) is list:
            text = ''
            for i in new_val['FoaInformation']:
                text = text + ' ' + i['Name']

            usa_dict['foa'] = text
        else:
            pass
    except:
        pass
    
    if usa_dict['framework_programme'] == 'FP4':
        usa_fp4.append(usa_dict)
    elif usa_dict['framework_programme'] == 'FP5':
        usa_fp5.append(usa_dict)
    elif usa_dict['framework_programme'] == 'FP6':
        usa_fp6.append(usa_dict)
    elif usa_dict['framework_programme'] == 'FP7':
        usa_fp7.append(usa_dict)
    if usa_dict['framework_programme'] == 'H2020':
        usa_h2020.append(usa_dict)


<p>
<ul>
<li>usa fp4: 39427 projects</li>
<li>usa fp5: 40210 projects</li>
<li>usa fp6: 55006 projects</li>
<li>usa fp7: 88668 projects</li>
<li>usa h2020: 35795 projects</li>
</ul>
</p>

In [4]:
df_fp4 = pd.DataFrame(usa_fp4)
df_fp5 = pd.DataFrame(usa_fp5)
df_fp6 = pd.DataFrame(usa_fp6)
df_fp7 = pd.DataFrame(usa_fp7)
df_h2020 = pd.DataFrame(usa_h2020)

# dfUSA.to_csv("usa_data.csv", sep = ';')

In [5]:
df_h2020.columns

Index([u'framework_programme', u'id', u'objective', u'state', u'subjects',
       u'title'],
      dtype='object')

In [6]:
df_fp4.to_pickle('usaFP4')
df_fp5.to_pickle('usaFP5')
df_fp6.to_pickle('usaFP6')
df_fp7.to_pickle('usaFP7')
df_h2020.to_pickle('usaH2020')